In [ ]:

!pip install gym
!pip install torch
!pip install transformers==4.20.0
!pip install keras_nlp==0.3.0
!pip install datasets
!pip install huggingface-hub
!pip install rouge-score
!pip install tensorflow
!pip install streamlit
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.4/142.4 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24932 sha256=c916ec7314d13f6f82d0b354a06a1ef6ec3870d8db7641227184d0444d324e16
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8

In [ ]:
%%writefile summarization1.py
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import torch
from torch.utils.data import DataLoader, TensorDataset
import streamlit as st

# Constants
MAX_TARGET_LENGTH = 256
MIN_TARGET_LENGTH = 5
LEARNING_RATE = 5e-4
BATCH_SIZE = 1

# Load the saved model and tokenizer
SAVE_PATH = "/Users/srikarkilari/InternshipDemo/summarization_files"
tokenizer = AutoTokenizer.from_pretrained(SAVE_PATH)
model = AutoModelForSeq2SeqLM.from_pretrained(SAVE_PATH)
summarization_pipeline = pipeline('summarization', model=model, tokenizer=tokenizer, framework="pt")

def fine_tune_model_on_single_example(model, input_text, corrected_summary, tokenizer, epochs=3, clip_value=1.0):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device).train()

    inputs = tokenizer([input_text], padding=True, truncation=True, return_tensors="pt")
    targets = tokenizer([corrected_summary], padding=True, truncation=True, return_tensors="pt")

    inputs = inputs.input_ids.to(device)
    targets = targets.input_ids.to(device)

    dataset = TensorDataset(inputs, targets)
    dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

    optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-4)

    for epoch in range(epochs):
        for input_batch, target_batch in dataloader:
            optimizer.zero_grad()
            outputs = model(input_ids=input_batch, labels=target_batch)
            loss = outputs.loss
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip_value)
            optimizer.step()

# Initialize session states
if 'step' not in st.session_state:
    st.session_state.step = 1
if 'input_text' not in st.session_state:
    st.session_state.input_text = ''
if 'initial_summary' not in st.session_state:
    st.session_state.initial_summary = ''
if 'corrected_summary' not in st.session_state:
    st.session_state.corrected_summary = ''

st.title('Text Summarization Chatbot')

if st.session_state.step == 1:
    st.session_state.input_text = st.text_area("Enter text to summarize (or type 'BYE' to exit):", st.session_state.input_text)
    if st.session_state.input_text.strip().upper() == 'BYE':
        st.write("Thank you! Goodbye!")
        st.session_state.step = 4
    elif st.button('Summarize'):
        st.write(f"**Provided Text:**\n{st.session_state.input_text}\n")  # Displaying the provided input text
        st.session_state.initial_summary = summarization_pipeline(st.session_state.input_text, max_length=MAX_TARGET_LENGTH, min_length=MIN_TARGET_LENGTH)[0]['summary_text']
        st.session_state.step = 2

elif st.session_state.step == 2:
    st.write(f"**Provided Text:**\n{st.session_state.input_text}\n")  # Displaying the provided input text
    st.write(f"**Initial Summary:** {st.session_state.initial_summary}")
    st.session_state.corrected_summary = st.text_area("Please provide a corrected summary (if needed):", st.session_state.corrected_summary)

    if st.button('Fine-tune and Regenerate Summary'):
        if st.session_state.corrected_summary:
            fine_tune_model_on_single_example(model, st.session_state.input_text, st.session_state.corrected_summary, tokenizer)
            refined_summary = summarization_pipeline(st.session_state.input_text, max_length=MAX_TARGET_LENGTH, min_length=MIN_TARGET_LENGTH)[0]['summary_text']
            st.write(f"**Refined Summary (based on feedback):** {refined_summary}")
            st.session_state.step = 3
        else:
            st.write("No corrected summary provided. Using initial summary.")
            st.session_state.step = 3

elif st.session_state.step == 3:
    if st.button('Continue with a new text'):
        st.session_state.step = 1
        st.session_state.input_text = ''
        st.session_state.initial_summary = ''
        st.session_state.corrected_summary = ''


Overwriting summarization1.py
